In [1]:
from pathlib import Path
import re

import natsort
import numpy as np
import pandas as pd

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
def read_experiment_file(fn):
    experiment_data = pd.DataFrame(
        columns=["gNB0", "P0", "gNB1", "P1", "gNB2", "P2", "gNB3", "P3"]
    )

    with open(fn) as f:
        contents = f.read()

        # Patterns to extract ToA and Power values
        bracket_pattern = r"\[(.*?)\]"
        toa_pattern = r"DL PRS ToA ==> (-?\d+\.\d+)"
        power_pattern = r"peak channel power (-?\d+\.\d+|-inf) dBm"

        # To keep track of the measurement number
        first_sfn = -1
        current_sfn = -1

        # Format: [Time] [gNBi] [rsc 0] [Rx 0] [sfn #] TOA Information
        for line in contents.split("\n\n"):
            if line == "":
                break

            # This returns all values within []
            metadata = re.findall(bracket_pattern, line)

            # First line shennanigans
            if first_sfn == -1:
                first_sfn = int(metadata[4][4:])
                current_sfn = first_sfn
                experiment_data.loc[0] = np.repeat(-np.inf, 8)

            # If we change measurement set (The 4 gNB were already measured or there is a missing value)
            if current_sfn != int(metadata[4][4:]):
                current_sfn = int(metadata[4][4:])
                experiment_data.loc[current_sfn - first_sfn] = np.repeat(-np.inf, 8)

            gNB = metadata[1].replace(" ", "")

            # Extracting values
            toa = re.findall(toa_pattern, line)[0]
            experiment_data.loc[current_sfn - first_sfn][gNB] = float(toa)
            power = re.findall(power_pattern, line)[0]
            experiment_data.loc[current_sfn - first_sfn][f"P{gNB[3]}"] = float(power)

    return experiment_data

In [21]:
df = read_experiment_file("../measurements/experiments/exp0_100mhz_0.txt")
df.head(5)

,gNB0,P0,gNB1,P1,gNB2,P2,gNB3,P3
0,-3.0,-54.5,-2.0,-55.3,-1.0,-53.4,-2.0,-57.1
1,-3.0,-55.3,-2.0,-55.1,-1.0,-52.7,-2.0,-57.5
2,-2.0,-56.0,-2.0,-55.3,-1.0,-52.5,-2.0,-57.7
3,-3.0,-56.0,-3.0,-55.0,-2.0,-52.3,-3.0,-57.8
4,-2.0,-55.6,-2.0,-55.3,-1.0,-52.5,-2.0,-58.0


In [41]:
ue_positions = np.arange(0, 6)
frequencies = np.array([40, 80, 100])

basepath = Path("../measurements/experiments/")

for ue in ue_positions:
    for frequency in frequencies:
        experiment_data = pd.DataFrame(
            columns=["gNB0", "P0", "gNB1", "P1", "gNB2", "P2", "gNB3", "P3"]
        )
        files = [fn for fn in basepath.rglob(f"exp{ue}_{frequency}mhz_[0-9].txt")]
        files = natsort.natsorted(files, key=str)
        for fn in files:
            df = read_experiment_file(fn)
            if experiment_data.empty:
                experiment_data = df
            else:
                experiment_data = pd.concat((experiment_data, df), ignore_index=True)

        experiment_data.to_csv(f"comnets_data/experiments/exp{ue}/{frequency}mhz.csv")
        print(f"Saved to: comnets_data/experiments/exp{ue}/{frequency}mhz.csv")

/var/folders/4n/v1n1ffhd0h74xtlw1g_pf4r00000gn/T/ipykernel_5229/2762371412.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  experiment_data = pd.concat((experiment_data,df), ignore_index=True)
/var/folders/4n/v1n1ffhd0h74xtlw1g_pf4r00000gn/T/ipykernel_5229/2762371412.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  experiment_data = pd.concat((experiment_data,df), ignore_index=True)
/var/folders/4n/v1n1ffhd0h74xtlw1g_pf4r00000gn/T/ipykernel_5229/2762371412.py:8: FutureWarning: The behavior

In [8]:
ue_positions = [0, 4]
frequencies = [40, 100]
oversampling = [1, 2, 4, 8, 16]

basepath = Path("../measurements/oversampling/")

for ue in ue_positions:
    for frequency in frequencies:
        for os in oversampling:
            experiment_data = pd.DataFrame(
                columns=["gNB0", "P0", "gNB1", "P1", "gNB2", "P2", "gNB3", "P3"]
            )
            files = [
                fn
                for fn in basepath.rglob(
                    f"exp{ue}_{frequency}mhz_newday_[0-9]_oversapling{os}.txt"
                )
            ]
            files = natsort.natsorted(files, key=str)
            for fn in files:
                df = read_experiment_file(fn)
                if experiment_data.empty:
                    experiment_data = df
                else:
                    experiment_data = pd.concat(
                        (experiment_data, df), ignore_index=True
                    )

            experiment_data.to_csv(
                f"comnets_data/oversampling/os{os}/exp{ue}_{frequency}mhz.csv"
            )
            print(
                f"Saved to : comnets_data/oversampling/os{os}/exp{ue}_{frequency}mhz.csv"
            )

Saved to : comnets_data/oversampling/os1/exp0_40mhz.csv
Saved to : comnets_data/oversampling/os2/exp0_40mhz.csv
Saved to : comnets_data/oversampling/os4/exp0_40mhz.csv
Saved to : comnets_data/oversampling/os8/exp0_40mhz.csv
Saved to : comnets_data/oversampling/os16/exp0_40mhz.csv
Saved to : comnets_data/oversampling/os1/exp0_100mhz.csv
Saved to : comnets_data/oversampling/os2/exp0_100mhz.csv
Saved to : comnets_data/oversampling/os4/exp0_100mhz.csv
Saved to : comnets_data/oversampling/os8/exp0_100mhz.csv
Saved to : comnets_data/oversampling/os16/exp0_100mhz.csv
Saved to : comnets_data/oversampling/os1/exp4_40mhz.csv
Saved to : comnets_data/oversampling/os2/exp4_40mhz.csv
Saved to : comnets_data/oversampling/os4/exp4_40mhz.csv
Saved to : comnets_data/oversampling/os8/exp4_40mhz.csv
Saved to : comnets_data/oversampling/os16/exp4_40mhz.csv
Saved to : comnets_data/oversampling/os1/exp4_100mhz.csv
Saved to : comnets_data/oversampling/os2/exp4_100mhz.csv
Saved to : comnets_data/oversampling/o

In [7]:
# Special case for position 5
ue_positions = [5]
frequencies = [40, 100]
oversampling = [2, 4, 8, 16]

basepath = Path("../measurements/oversampling/")

for ue in ue_positions:
    for frequency in frequencies:
        for os in oversampling:
            experiment_data = pd.DataFrame(
                columns=["gNB0", "P0", "gNB1", "P1", "gNB2", "P2", "gNB3", "P3"]
            )
            files = [
                fn
                for fn in basepath.rglob(
                    f"exp{ue}_{frequency}mhz_oversampling{os}_[0-9].txt"
                )
            ]
            files = natsort.natsorted(files, key=str)
            for fn in files:
                df = read_experiment_file(fn)
                if experiment_data.empty:
                    experiment_data = df
                else:
                    experiment_data = pd.concat(
                        (experiment_data, df), ignore_index=True
                    )

            experiment_data.to_csv(
                f"comnets_data/oversampling/os{os}/exp{ue}_{frequency}mhz.csv"
            )

/var/folders/4n/v1n1ffhd0h74xtlw1g_pf4r00000gn/T/ipykernel_42221/4236737211.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  experiment_data = pd.concat((experiment_data,df), ignore_index=True)


In [4]:
# Special case for position 5
ue_positions = [4]
frequencies = [100]
oversampling = [1, 2, 4, 8]

basepath = Path("../measurements/cir/")

for ue in ue_positions:
    for frequency in frequencies:
        for os in oversampling:
            experiment_data = pd.DataFrame(
                columns=["gNB0", "P0", "gNB1", "P1", "gNB2", "P2", "gNB3", "P3"]
            )
            files = [
                fn
                for fn in basepath.rglob(
                    f"exp{ue}_{frequency}mhz_newday_oversampling{os}_csi_[0-9].txt"
                )
            ]
            files = natsort.natsorted(files, key=str)
            for fn in files:
                df = read_experiment_file(fn)
                if experiment_data.empty:
                    experiment_data = df
                else:
                    experiment_data = pd.concat(
                        (experiment_data, df), ignore_index=True
                    )

            filename = f"comnets_data/oversampling/os{os}/exp{ue}_{frequency}mhz.csv"
            if Path(filename).is_file():
                d = pd.read_csv(filename)
                if experiment_data.empty:
                    experiment_data = d
                else:
                    experiment_data = pd.concat((d, experiment_data), ignore_index=True)

            experiment_data.to_csv(filename)
            print(f"Saved to: {filename}")

/var/folders/4n/v1n1ffhd0h74xtlw1g_pf4r00000gn/T/ipykernel_11881/292972434.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  experiment_data = pd.concat((experiment_data, df), ignore_index=True)


Saved to: comnets_data/oversampling/os1/exp4_100mhz.csv
Saved to: comnets_data/oversampling/os2/exp4_100mhz.csv


/var/folders/4n/v1n1ffhd0h74xtlw1g_pf4r00000gn/T/ipykernel_11881/292972434.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  experiment_data = pd.concat((experiment_data, df), ignore_index=True)


Saved to: comnets_data/oversampling/os4/exp4_100mhz.csv
Saved to: comnets_data/oversampling/os8/exp4_100mhz.csv


/var/folders/4n/v1n1ffhd0h74xtlw1g_pf4r00000gn/T/ipykernel_11881/292972434.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  experiment_data = pd.concat((experiment_data, df), ignore_index=True)
/var/folders/4n/v1n1ffhd0h74xtlw1g_pf4r00000gn/T/ipykernel_11881/292972434.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  experiment_data = pd.concat((experiment_data, df), ignore_index=True)
